# The goal of this program is to pull data from Riot's API and run some analytics on the players in challenger league, namely: 

## *Champion picks of every current Challenger player (including their climb games) **from season start***

In [1]:
import pandas as pd
import os
import cassiopeia as cass
import requests
import random
from ratelimit import limits
import json
import datetime
pwd = os.getcwd()
pd.options.display.max_rows = 10
cass.set_riot_api_key("REDACTED")

key = 'REDACTED'

### Creating a list of all players currently in Challenger League, and their puuids
 

In [ ]:
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives, region="NA")

summoner_name = []

for item in challenger_league:
    summoner_name.append(item.summoner.name)

summoner_name


In [ ]:
summoner_puuid = []  

for item in summoner_name:
    bucket = cass.Summoner(name=item, region='NA')
    summoner_puuid.append(bucket.puuid)
    
summoner_puuid


## Now that I have a list of summoner names and puuids, I can pull match histories, and create useful lists that will be used later

In [ ]:
total_match_list = []

for item in summoner_name:
    bucket = cass.Summoner(name=item, region='NA')
    challenger_history = (cass.get_match_history(region=cass.data.Region.north_america,
                                             queue=cass.data.Queue.ranked_solo_fives, puuid=bucket.puuid,
                                             continent=cass.data.Continent.americas))
    total_match_list.append(challenger_history)


total_match_list

In [ ]:

champion_picks = []
data = []
matches = []

for position, match_list in enumerate(total_match_list):
    bucket = cass.Summoner(name=summoner_name[position], region='NA')
    for match in match_list:
        champion = match.participants[bucket].champion.name
        champion_picks.append(champion)
        matches.append(match)
        tup = (bucket.name, champion, match)
        data.append(tup)

champion_picks
data        
matches

        


In [ ]:
data.sort

data

In [7]:
dataframe = pd.DataFrame(data, columns= ['Summoner', 'Champion', 'Match'])

dataframe

,Summoner,Champion,Match
0,rCadeKing,Anivia,"Match(id=4224559581, region='AMERICAS')"
1,rCadeKing,Viktor,"Match(id=4224526525, region='AMERICAS')"
2,rCadeKing,Vex,"Match(id=4224522522, region='AMERICAS')"
3,rCadeKing,Kayle,"Match(id=4223947368, region='AMERICAS')"
4,rCadeKing,Viktor,"Match(id=4223944060, region='AMERICAS')"
...,...,...,...
5995,Pobelter,Viktor,"Match(id=4219722827, region='AMERICAS')"
5996,Pobelter,Ahri,"Match(id=4219678077, region='AMERICAS')"
5997,Pobelter,Orianna,"Match(id=4219664608, region='AMERICAS')"
5998,Pobelter,Viktor,"Match(id=4219650264, region='AMERICAS')"


## I have pulled matchlists, sorted them by summoner, and stored them into a data frame

### Now I can find which champions have the most picks, and test the things like:

#### "Most one trick", "Least one trick", "Highest Pick Rate for Climb", "Lowest Pick Rate for Climb" etc...

In [8]:
dataframe

dataframe_modified = dataframe.copy()

matches_set = set(matches)

match_count = len(matches_set)

match_count


2916

In [9]:
dataframe_modified.describe()

,Summoner,Champion,Match
count,6000,6000,6000
unique,300,157,2916
top,rCadeKing,Jinx,"Match(id=4225982391, region='AMERICAS')"
freq,20,192,10


# Most Picked Champions

In [10]:
from collections import Counter

In [11]:
champ_pick_num = Counter(elem[1] for elem in data)

sorted_10  = sorted(champ_pick_num, key=champ_pick_num.get, reverse=True)[:10]

sorted_10

for item in sorted_10:
    print(item + ": picked in " + str(100*((champ_pick_num[item]/(match_count))))  +"% of games\n")

Jinx: picked in 6.584362139917696% of games

Ahri: picked in 5.246913580246913% of games

Graves: picked in 4.938271604938271% of games

Aphelios: picked in 4.938271604938271% of games

Jhin: picked in 4.561042524005487% of games

Zeri: picked in 4.38957475994513% of games

Viego: picked in 3.943758573388203% of games

Lee Sin: picked in 3.7379972565157753% of games

Gwen: picked in 3.60082304526749% of games

Qiyana: picked in 3.3607681755829906% of games



# Least Picked Champions 
##### *With at least one pick*

In [12]:
sorted_10_bottom  = sorted(champ_pick_num, key=champ_pick_num.get, reverse=False)[:10]

sorted_10_bottom

for item in sorted_10_bottom:
    print(item + ": picked in " + str(100*((champ_pick_num[item]/(match_count))))  +"% of games\n")

Singed: picked in 0.034293552812071325% of games

Rammus: picked in 0.034293552812071325% of games

Warwick: picked in 0.034293552812071325% of games

Skarner: picked in 0.034293552812071325% of games

Sivir: picked in 0.034293552812071325% of games

Kled: picked in 0.034293552812071325% of games

Elise: picked in 0.06858710562414265% of games

Azir: picked in 0.06858710562414265% of games

Illaoi: picked in 0.06858710562414265% of games

Nasus: picked in 0.06858710562414265% of games



# Champions Not Picked at All

In [13]:
picked_champs = sorted(champ_pick_num, key=champ_pick_num.get, reverse=False)

full_champ_list = []

list = cass.get_champions(region='NA')

for item in list:
    full_champ_list.append(item.name)

champions_not_picked = set(full_champ_list).difference(set(picked_champs))


print("These Champions were picked in 0 games:") 
for item in champions_not_picked:
    print('\n' + item)

These Champions were picked in 0 games:

Yorick

Taric


# Most one-trick player

In [22]:
picks_aggregated_to_summoner = {}

for name in summoner_name:
    history_list = []
    for tuple in data:
        if name == tuple[0]:
            history_list.append(tuple[1])
    picks_aggregated_to_summoner.update({(int(summoner_name.index(str(name)))):history_list})


picks_aggregated_to_summoner

{0: ['Anivia',
  'Viktor',
  'Vex',
  'Kayle',
  'Viktor',
  'Kayle',
  'Tryndamere',
  'Sett',
  'Blitzcrank',
  'Qiyana',
  'Qiyana',
  'Kayle',
  'Cassiopeia',
  'Gwen',
  'Kassadin',
  'Vex',
  'Gwen',
  'Qiyana',
  'Ezreal',
  'Viktor'],
 1: ['Viego',
  'Kindred',
  'Xin Zhao',
  'Xin Zhao',
  'Xin Zhao',
  'Ekko',
  'Kindred',
  'Viego',
  'Viego',
  'Viego',
  'Kindred',
  'Viego',
  'Kayn',
  'Ekko',
  'Kayn',
  'Zed',
  'Kayn',
  'Kindred',
  'Ekko',
  'Viego'],
 2: ['Riven',
  'Zeri',
  'Graves',
  'Gwen',
  'Jayce',
  'Qiyana',
  'Tristana',
  'Gwen',
  'Tristana',
  'Lucian',
  'Irelia',
  'Vayne',
  'Aatrox',
  'Aatrox',
  'Vayne',
  'Lucian',
  'Irelia',
  'Ryze',
  'Aatrox',
  'Irelia'],
 3: ['Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger',
  'Heimerdinger

In [24]:
for key, list in picks_aggregated_to_summoner.items():
    print(list)

['Anivia', 'Viktor', 'Vex', 'Kayle', 'Viktor', 'Kayle', 'Tryndamere', 'Sett', 'Blitzcrank', 'Qiyana', 'Qiyana', 'Kayle', 'Cassiopeia', 'Gwen', 'Kassadin', 'Vex', 'Gwen', 'Qiyana', 'Ezreal', 'Viktor']
['Viego', 'Kindred', 'Xin Zhao', 'Xin Zhao', 'Xin Zhao', 'Ekko', 'Kindred', 'Viego', 'Viego', 'Viego', 'Kindred', 'Viego', 'Kayn', 'Ekko', 'Kayn', 'Zed', 'Kayn', 'Kindred', 'Ekko', 'Viego']
['Riven', 'Zeri', 'Graves', 'Gwen', 'Jayce', 'Qiyana', 'Tristana', 'Gwen', 'Tristana', 'Lucian', 'Irelia', 'Vayne', 'Aatrox', 'Aatrox', 'Vayne', 'Lucian', 'Irelia', 'Ryze', 'Aatrox', 'Irelia']
['Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger', 'Heimerdinger']
['Vex', 'Viktor', 'Yone', 'Viktor', 'Ahri', 'Swain', 'Xerath', 'Ahri', 'Xerath', 'Sylas', 'Varus

In [25]:


import statistics 

mode_tup_list = []
modes = []
most_one_trick = []


for key, list in picks_aggregated_to_summoner.items():
    mode_champion = statistics.mode(list)
    mode_value = list.count(mode_champion)
    pick_percentage = (mode_value / len(list)) * 100
    mode_tup_list.append((summoner_name[key], mode_champion, mode_value, pick_percentage))
    modes.append(int(round(pick_percentage)))



largest_one_trick_value = max(modes)

for i, mode, in enumerate(modes):
    if mode == largest_one_trick_value:
        location = i
        most_one_trick.append((summoner_name[location], mode_tup_list[location][1], largest_one_trick_value))

most_one_trick





StatisticsError: no mode for empty data

# Least one-trick player



In [ ]:
least_one_trick = []

smallest_one_trick_value = min(modes)

for i, mode, in enumerate(modes):
    if mode == smallest_one_trick_value:
        location = i
        least_one_trick.append((summoner_name[location], mode_tup_list[location][1], smallest_one_trick_value))

least_one_trick

[('WX Rika19981209', 'Hecarim', 10),
 ('NøticeMeSenpai', 'Sylas', 10),
 ('From Iron', 'Seraphine', 10),
 ('yticanet001', 'Aatrox', 10)]

# Least Games Played


In [ ]:
games_played = []

for key, list in picks_aggregated_to_summoner.items():
    games_played.append(len(list))


min_games_played = min(games_played)
least_games_played = []

for i, game in enumerate(games_played):
    location = i
    if game == min_games_played:
        least_games_played.append((summoner_name[location], game))
        

least_games_played

    


# Most Games Played


In [ ]:
max_games_played = min(games_played)
most_games_played = []

for i, game in enumerate(games_played):
    location = i
    if game == max_games_played:
        most_games_played.append((summoner_name[location], game))
        

most_games_played